In [1]:
from numpy import zeros, max, sqrt, isnan, isinf, dot, diag, count_nonzero, where
from numpy.linalg import svd, linalg, LinAlgError, norm
from scipy.linalg import svd as scipy_svd


class FrequentDirections(object):
    
    def __init__(self , rows, columns, op='fd'):
        """
        """
        self.class_name = 'FrequentDirections'
        self.op = op
        self.columns = columns
        self.rows = rows
        self.sketchMatrix = zeros((self.rows, self.columns)) 
        self.S = np.zeros(self.columns)
        self.U = []
        self.Vt = []
        self.step = 1
        self.nextZeroRow = 0
        self.emptyRows = self.rows
        if self.op == 'fd':
            print("Matrix Sketching Using Frequent Direction")
            self.reduceRank = self.__FDOperate__
        elif self.op == 'ssd':
            print("Matrix Sketching Using Space Saving Direction")
            self.op = 2
            self.reduceRank = self.__SSDOperate__
        elif self.op == 'cfd':
            print("Matrix Sketching Using Compensative Freqent Direction")
            self.reduceRank = self.__CFDperate__
        elif self.op == 'isvd':
            print("Matrix Sketching Using iSVD")
            self.reduceRank = self.__iSVDOperate__
        elif type(self.op) != str and self.op > 0 and self.op < 1:
            print("Matrix Sketching Using Parameterized Frequent Direction")
            self.reduceRank = self.__PFDOperate__
            self.DELTA = 0
        else:
            print("Type of Reduce Rank algorithm is not correct")
            raise ValueError
    
    # Add new vector to the sketch matrix
    def add(self,vector):     
        if count_nonzero(vector) == 0:
            return
        
        # If the approximate matrix is full, call the operate method to free half of the columns
#         if self.nextZeroRow >= self.rows:
        if self.emptyRows <= 0:
            try:
                [self.U,self.S,self.Vt] = svd(self.sketchMatrix , full_matrices=True)
            except LinAlgError as err:
                [self.U,self.S,self.Vt] = scipy_svd(self.sketchMatrix , full_matrices = True)
            self.reduceRank()
#             Push the new vector to the next zero row
#             self.sketchMatrix[where(self.S == 0)[0][0],:] = vector
        # Push the new vector to the next zero row and increase the next zero row index
#         else:    
        self.sketchMatrix[self.nextZeroRow,:] = vector
        self.nextZeroRow += 1
        self.emptyRows -= 1


    # Shrink the approximate matrix using Frequent Direction
    def __FDOperate__(self):
        if len(self.S) >= self.rows:
            # Calculating matrix s
            delta = sqrt(self.S[:]**2 - self.S[len(self.S)-1]**2)
            self.S = delta
            self.S[len(self.S)-1] = 0
            print (self.S)
            #Shrink the sketch matrix
            self.sketchMatrix[:,:] = dot(diag(self.S), self.Vt[:self.rows,:])
            self.nextZeroRow = (len(self.S)-1)
            self.emptyRows += 1
        else:
#             s = zeros((self.rows,self.columns))
#             s[:len(self.S), :len(self.S)] = np.diag(self.S)
            self.sketchMatrix[:len(self.S),:] = dot(diag(self.S), self.Vt[:len(self.S),:])
#             self.sketchMatrix[len(self.S):,:] = 0
            self.nextZeroRow = len(self.S)-1
            self.emptyRows += 1
            
        
    # Shrink the approximate matrix using iterative SVD
    def __iSVDOperate__(self):
        # Calculating matrix s
        self.S[len(self.S)-1] = 0
        print (self.S)
        #Shrink the sketch matrix
        self.sketchMatrix[:len(self.S),:] = dot(diag(self.S), self.Vt[:len(self.S),:])
        self.nextZeroRow = (len(self.S)-1)
        self.emptyRows += 1


   	# Shrink the approximate matrix using Parameterized FD
    def __PFDOperate__(self):
        #Shrink the sketch matrix
        if len(self.S) >= self.rows:
            # Calculating matrix s
#             delta = sqrt(self.S[:round(len(self.S)*(1-self.op))]**2 - self.S[-1]**2)
            delta = self.S[-1]**2
            print (self.S)
            self.S[round(len(self.S)*(1-self.op)):] = sqrt(self.S[round(len(self.S)*(1-self.op)):]**2 - self.S[-1]**2)
            print (self.S)
            #Shrink the sketch matrix
            self.sketchMatrix[:,:] = dot(diag(self.S), self.Vt[:self.rows,:])
            self.nextZeroRow = len(self.S) - 1
            self.emptyRows += 1
        else:
            self.sketchMatrix[:len(self.S),:] = dot(diag(self.S), Vt[:len(self.S),:])
            self.sketchMatrix[int(len(self.S)/2):,:] = 0
            self.nextZeroRow = int(len(self.S)/2)
            self.emptyRows += 1

    # Shrink the approximate matrix using Space Saving Direction
    def __SSDOperate__(self):
        # Calculating matrix s
        self.S[-1] = sqrt(self.S[-1]**2 + self.S[-2]**2)
        self.S[len(self.S)-2] = 0
        print (self.S)
        #Shrink the sketch matrix
        self.sketchMatrix[:len(self.S),:] = dot(diag(self.S), self.Vt[:len(self.S),:])
        self.nextZeroRow = len(self.S)-2
        self.emptyRows += 1
        
        
    # Shrink the approximate matrix using Compensative Direction
    def __CFDperate__(self):
        # Calculating SVD
        try:
            [U,self.S,Vt] = svd(self.sketchMatrix, full_matrices=False)
        except LinAlgError as err:
            [U,self.S,Vt] = scipy_svd(self.sketchMatrix, full_matrices = False)
        # Calculating matrix s
        self.DELTA += self.S[-1]**2
        self.S = sqrt(self.S[:len(self.S)]**2 + self.DELTA)
        delta = sqrt(self.S[:]**2 - self.S[len(self.S)-1]**2)
        self.S = delta
        self.S[len(self.S)-1] = 0
        print (self.S)
        #Shrink the sketch matrix
        self.sketchMatrix[:len(self.S),:] = dot(diag(self.S), self.Vt[:len(self.S),:])
        self.nextZeroRow = len(self.S) - 1
        self.emptyRows += 1


    # Return the sketch matrix
    def getSketchMatrix(self):
        return self.sketchMatrix[:self.rows,:]
    
    # Return the S matrix
    def getS(self):
        return self.S
    
    # Return the U matrix
    def getU(self):
        return self.U
    
    # Return the Vt matrix
    def getVt(self):
        return self.Vt

In [2]:
import numpy as np
A = 100* np.random.rand(1000,20)
A

array([[ 83.74084176,  91.23513739,  60.74703525, ...,  76.6943409 ,
          6.97543416,  54.98656818],
       [ 72.55893842,  83.5959818 ,  29.07869358, ...,  84.83877644,
         60.38405519,  54.08752997],
       [ 13.56377261,  20.54039477,  90.26751383, ...,  88.36595531,
         51.14041503,  23.76052143],
       ..., 
       [ 63.1151242 ,  41.22282927,  54.73688083, ...,  90.28614919,
         72.46784912,  26.10402741],
       [ 88.60464666,  92.99555729,  87.76846618, ...,  81.23027565,
         47.12179455,  25.83428603],
       [ 81.3738737 ,  18.882511  ,  29.44722171, ...,  57.99935228,
         71.48411714,  74.65780134]])

In [3]:
l = 10
fd =  FrequentDirections(l,20,op=0.8)
for i in range(1000):
    row = A[i,:]
    fd.add(row)

Matrix Sketching Using Parameterized Frequent Direction
[ 702.58123966  179.67302138  174.24305563  140.18644401  124.00335813
  112.86924637  109.21434726   91.3560306    73.37137798   40.11199857]
[ 702.58123966  179.67302138  169.56317408  134.32522718  117.33652628
  105.50115804  101.58150038   82.07893699   61.43603729    0.        ]
[ 736.18132066  181.25443662  170.47467652  137.74866619  132.8201833
  117.30642538  104.96114565   96.60596933   62.46911085   42.29564282]
[ 736.18132066  181.25443662  165.14446383  131.09452176  125.9058366
  109.41606845   96.06206689   86.85500509   45.97247447    0.        ]
[ 747.319072    181.94354836  165.16000237  137.2091363   128.30179601
  114.22810698   96.39863019   88.49676868   86.34936699   45.88719967]
[ 747.319072    181.94354836  158.65746528  129.3085921   119.81534028
  104.60604825   84.77653454   75.6706216    73.1476458     0.        ]
[ 795.98014619  195.53024685  165.23482807  131.23219585  121.89780113
  105.63749777   

[ 2502.36192538   351.19915303   182.93370339   147.17013365   121.02697361
   101.24060673    99.07506307    88.03237003    48.729803      14.14156373]
[ 2502.36192538   351.19915303   182.38628241   146.48912729   120.19793891
   100.24807543    98.06061542    86.88909223    46.63271251     0.        ]
[ 2510.1735004    352.56055212   183.13454206   162.41797751   124.80018966
   105.34508154   100.15084827    96.53097099    84.83837697    39.28634891]
[ 2510.1735004    352.56055212   178.87102416   157.59499423   118.45535078
    97.74542953    92.1236951     88.17488956    75.19396915     0.        ]
[ 2518.79736832   354.83006072   180.78463326   157.68662355   119.05448862
   107.18890717    95.60457054    88.17862922    79.92569179    56.63959534]
[ 2518.79736832   354.83006072   171.68296323   147.16326813   104.71832457
    91.002297      77.02071247    67.58274108    56.39213108     0.        ]
[ 2525.06378751   355.04340429   176.27680575   150.45049694   143.21867732
    98

   130.15928848    98.49224886    83.53467229    66.23064482    53.2252932 ]
[ 3384.86130744   464.05878245   168.41773058   153.8740201    139.29292579
   118.77924288    82.87213796    64.38252587    39.41530764     0.        ]
[ 3389.5137354    469.23655243   168.43814333   155.46020762   140.37790694
   118.79201623    91.8287664     81.42703922    62.19112042    39.25556948]
[ 3389.5137354    469.23655243   163.79990351   150.42232686   134.77743513
   112.11843463    83.01519501    71.33977138    48.23624906     0.        ]
[ 3395.48477902   469.25371759   174.90464325   153.4441388    137.35253523
   125.24537627   104.06351321    82.81748893    71.32845171    48.20136362]
[ 3395.48477902   469.25371759   168.13168284   145.67680761   128.61705749
   115.59858486    92.22712902    67.34511874    52.57733892     0.        ]
[ 3403.35015083   470.75505373   170.2949938    156.77026711   137.48422117
   115.9246998    113.45503346    88.98102442    66.90542144    49.96219806]
[ 340

   141.42184623   110.64942412    83.43979157    63.83950043    46.59970562]
[ 4101.13983493   555.18996353   176.45622427   158.70969247   143.83009046
   133.52380323   100.35817103    69.21463901    43.63426694     0.        ]
[ 4109.0662671    555.40623283   190.40040598   174.27793579   143.92286567
   137.01626294   100.39111196    93.19145492    62.97195516    42.73216585]
[ 4109.0662671    555.40623283   185.54319335   168.95786725   137.43272269
   130.18225037    90.84237647    82.81672097    46.2539635      0.        ]
[ 4113.91053069   556.01002146   191.66163306   169.95612897   145.25100119
   135.3426117    103.2108528     87.86345819    72.23537131    37.52154076]
[ 4113.91053069   556.01002146   187.95296104   165.76254026   140.32101527
   130.03751966    96.14891634    79.44885942    61.72586854     0.        ]
[ 4117.71260362   557.06561968   188.03066077   169.26102622   154.34747391
   134.55500032   117.38069715    93.02695252    78.8119737     61.55864787]
[ 411

[ 4716.96693932   629.44348986   172.11641545   152.42172546   111.86773856
   100.510807      91.42558514    79.257062      57.54488311    46.31829218]
[ 4716.96693932   629.44348986   165.7669336    145.21362953   101.82831992
    89.20223166    78.82419316    64.31405512    34.14717238     0.        ]
[ 4721.9958503    631.51558932   167.72644113   152.64566651   111.94789281
    99.71739628    87.77276834    73.86240997    61.9327228     34.14709494]
[ 4721.9958503    631.51558932   164.21368689   148.77726779   106.61288201
    93.68849998    80.85811505    65.49527856    51.66854033     0.        ]
[ 4726.60336731   631.51794689   170.05169229   155.58719794   109.4633814
   102.84221369    89.27486303    66.00693308    62.93890498    48.732538  ]
[ 4726.60336731   631.51794689   162.91935977   147.75830231    98.01720057
    90.56302036    74.80067452    44.52027577    39.83020839     0.        ]
[ 4732.6550633    631.6282748    172.16941362   148.84162487   123.73383943
    97.

...

[ 5577.83739493   747.57194589   190.71263918   186.55245921   135.12789728
   126.10856825    90.57125746    81.91318794    39.61754753     0.        ]
[ 5584.75441013   747.58079276   193.51065205   187.48010848   141.57803383
   126.1422573    104.05520505    87.75466888    77.64179517    39.20323032]
[ 5584.75441013   747.58079276   189.49796619   183.33547886   136.04207583
   119.8956872     96.38771929    78.51107337    67.01757299     0.        ]
[ 5587.28382456   747.6124698    201.09672242   185.88499173   136.26929873
   121.07523331   110.54380556    93.02157729    72.11229054    61.21926222]
[ 5587.28382456   747.6124698    191.55180422   175.51476315   121.74368037
   104.45771419    92.04420069    70.03724562    38.11016112     0.        ]
[ 5591.01918071   747.61983361   191.56161385   177.0350235    128.95181418
   117.37101038    93.33045163    91.68581208    68.83079887    37.89393706]
[ 5591.01918071   747.61983361   187.7762004    172.93192037   123.25834623
   111

   110.37779467   104.54550479    84.60264116    66.62637055    34.17703124]
[ 5958.66971875   799.08273277   183.89688388   156.912091     134.47579456
   104.95326623    98.8012809     77.39210183    57.19268999     0.        ]
[ 5960.87651551   799.29410182   187.80035915   169.87575755   150.91762726
   116.86383843   100.7477596     83.58976047    77.28892161    57.05918726]
[ 5960.87651551   799.29410182   178.92239671   160.0063191    139.71535122
   101.98728294    83.03228417    61.08598207    52.13277811     0.        ]
[ 5966.34935831   799.31523922   185.47993314   160.07828727   148.00431224
   104.29474437    97.55972942    82.79493405    60.37381356    50.61739767]
[ 5966.34935831   799.31523922   178.43958263   151.86486463   139.07967319
    91.18811741    83.40131808    65.52007446    32.90708764     0.        ]
[ 5969.99499804   800.19459656   188.56951939   162.05626951   144.96025832
   131.15934617    89.38276177    68.33739265    62.04667126    27.87675495]
[ 596

[ 6349.49945399   847.69129853   178.63735742   140.9636577    133.61176183
   113.4037795    107.78608447    78.32921591    52.83442403    50.18942521]
[ 6349.49945399   847.69129853   171.44190579   131.72613404   123.82699422
   101.69286505    95.38795312    60.13724022    16.50751221     0.        ]
[ 6353.82649068   848.46742398   181.45736473   145.82680724   124.96688707
   121.45300885    99.74162874    87.37700038    59.51736725    16.47088693]
[ 6353.82649068   848.46742398   180.7082873    144.89364235   123.87668364
   120.33097375    98.37226432    85.8105476     57.19289194     0.        ]
[ 6358.04473539   848.81143098   180.70892076   169.29895937   126.34197537
   120.59718599   111.18996125    97.83205963    83.74090525    55.26050674]
[ 6358.04473539   848.81143098   172.05228984   160.02629171   113.61589297
   107.19122009    96.48566669    80.73034303    62.91911956     0.        ]
[ 6362.0854149    851.1238461    183.30328409   161.33145593   125.24089444
   109

   111.55676184    76.23890287    46.89703851    14.71986199     0.        ]
[ 6735.02650021   886.07636294   185.42242883   168.60334932   136.94721353
   126.3955366     91.7130688     67.70040954    45.46534047    14.70868994]
[ 6735.02650021   886.07636294   184.83812256   167.96054251   136.15503565
   125.53679187    90.52591578    66.08327998    43.02036291     0.        ]
[ 6739.50048376   886.7781433    185.05359232   177.42017339   154.02046409
   135.07067743   101.4070155     89.36797102    64.33003238    32.10143203]
[ 6739.50048376   886.7781433    182.24799064   174.49187943   150.63798134
   131.20055626    96.19189599    83.40343102    55.74810425     0.        ]
[ 6743.82133409   888.04581329   197.04387704   174.519094     150.9173067
   135.55145483   118.11567852    95.01795633    65.56699359    52.88175482]
[ 6743.82133409   888.04581329   189.81519825   166.3142633    141.34904835
   124.81072435   105.61644531    78.94258694    38.7627483      0.        ]
[ 6748

[ 7091.62152092   938.68273915   176.95501465   174.48475175   155.23916257
   117.30967617   113.06221368    86.32823465    69.18134913    37.94175662]
[ 7091.62152092   938.68273915   172.83952185   170.30957606   150.53112868
   111.00442886   106.50580861    77.54345364    57.84878713     0.        ]
[ 7096.20457594   938.79822623   176.66597982   170.82737688   150.53294245
   120.66878082   109.57855286    98.77010835    66.36976532    51.90234085]
[ 7096.20457594   938.79822623   168.86981802   162.75177328   141.30220726
   108.93622758    96.507027      84.03381056    41.36535704     0.        ]
[ 7099.72994693   940.04152175   176.10621154   164.3736716    145.40665357
   111.87713317   108.77718205    84.06409883    72.70168839    35.67179162]
[ 7099.72994693   940.04152175   172.45556246   160.45630931   140.96318025
   106.03780557   102.7618539     76.12027322    63.34870778     0.        ]
[ 7103.59854415   940.053243     175.26044762   160.70350016   140.98329343
   120

In [4]:
B = fd.getSketchMatrix()
B


array([[  1.60623359e+03,   1.59075713e+03,   1.61517919e+03,
          1.61630924e+03,   1.57745144e+03,   1.62483185e+03,
          1.60334508e+03,   1.59497500e+03,   1.59112573e+03,
          1.62277272e+03,   1.60414091e+03,   1.60170897e+03,
          1.61561775e+03,   1.58566492e+03,   1.64197036e+03,
          1.60709134e+03,   1.60513331e+03,   1.53721589e+03,
          1.60195495e+03,   1.60639108e+03],
       [ -2.74521347e+02,   1.36021698e+02,   6.50847549e+01,
          3.68488444e+02,   1.54829814e+02,  -1.44148016e+02,
         -3.16431981e+01,  -4.31302783e+02,  -3.03800179e+02,
          7.11663877e+01,   3.35975489e+02,   3.34398030e+02,
         -5.92581430e+01,  -6.78884961e+01,  -4.34331670e+01,
          1.40099586e+02,  -2.27029244e+02,   1.18989826e+02,
         -6.57683809e+01,  -7.33138737e+01],
       [  4.68576164e-01,  -1.10999537e+01,  -3.57663506e+01,
         -2.83221664e+01,   4.69676855e+01,   3.73537019e+01,
          1.95588142e+01,  -1.76887851e+01

In [5]:
approxCovarianceMatrixB = dot(B.transpose(),B)
approxCovarianceMatrixA = dot(A.transpose(),A)
testMatrix = approxCovarianceMatrixA - approxCovarianceMatrixB

In [6]:
norm(testMatrix,ord=2)

1032331.3756871088

In [7]:
[U,s,Vt] = svd(A , full_matrices=False)
# Calculate Ak
for k in range(l-1,1,-1):
    #k = 1
    s = s[:k]
    U = U[:,:k]
    Vt = Vt[:k,:]
    Ak = dot(dot(U,diag(s)),Vt)
    print((norm((A-Ak),ord='fro'))**2/(l-k))

8239246.83379
4553495.79071
3336529.48569
2730114.26031
2370890.30816
2132553.97891
1966663.93192
1849151.74065


In [46]:
a = np.random.randn(6, 9)

In [147]:
U, s, V = np.linalg.svd(a, full_matrices=True)
s[len(s)-1]=0

In [148]:
S = np.zeros((6, 9))

In [149]:
S[:6, :6] = np.diag(s)

In [133]:
S

array([[ 5.27960421,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  3.49119658,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  3.07571292,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  2.13285343,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  1.71940022,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ]])

In [134]:
dot(U,S)

array([[-2.111955  , -1.83912717,  0.43684514, -0.53553675,  0.84320727,
         0.        ,  0.        ,  0.        ,  0.        ],
       [-4.05462681, -0.10887756, -0.6191263 ,  1.20598194, -0.25647879,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.96191504, -2.1482494 ,  0.76071578,  0.37585904, -1.16781318,
         0.        ,  0.        ,  0.        ,  0.        ],
       [-1.62156151,  1.12767506,  2.66250246, -0.33203303, -0.22187559,
         0.        ,  0.        ,  0.        ,  0.        ],
       [-1.74412757,  0.78066015, -1.09400233, -1.32298585, -0.87483109,
         0.        ,  0.        ,  0.        ,  0.        ],
       [-0.61412124, -1.51596625, -0.14632149, -0.89781077,  0.03480553,
         0.        ,  0.        ,  0.        ,  0.        ]])

In [121]:
type(0.2)

float

In [35]:
a = [1,2,3,4,5,6,7,8]

In [37]:
a[4:]

[5, 6, 7, 8]